<a href="https://colab.research.google.com/github/dikaaaa21/TUGAS-DASAR-ALGORITMA-DAN-PEMROGRAMAN-PERTEMUAN-14/blob/main/TUGAS_DASAR_ALGORITMA_DAN_PEMROGRAMAN_PERTEMUAN_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import os
import time

# Nama file untuk menyimpan data inventori [cite: 33, 45]
NAMA_FILE = 'inventori.json'

class TokoOnline:
    """
    Kelas utama yang mengelola semua logika untuk inventori dan belanja.
    """
    def __init__(self, file_path):
        """
        Inisialisasi TokoOnline.
        Memuat data inventori dari file jika ada.
        """
        self.file_path = file_path
        self.inventori = self.muat_data()

    def muat_data(self):
        """
        Memuat data inventori dari file JSON.
        Jika file tidak ada, kembalikan dictionary kosong.
        """
        try:
            with open(self.file_path, 'r') as f:
                return json.load(f)
        except FileNotFoundError:
            return {}
        except json.JSONDecodeError:
            return {}

    def simpan_data(self):
        """
        Menyimpan data inventori saat ini ke dalam file JSON.
        """
        with open(self.file_path, 'w') as f:
            json.dump(self.inventori, f, indent=4)

    # ===============================================
    # FUNGSI-FUNGSI ADMIN [cite: 36]
    # ===============================================

    def tambah_barang(self):
        """
        Menambahkan barang baru ke dalam inventori. [cite: 37]
        Kode barang dibuat unik dan otomatis.
        """
        try:
            nama = input("Masukkan nama barang: ")
            harga = int(input("Masukkan harga barang: "))
            stok = int(input("Masukkan jumlah stok: "))

            if harga <= 0 or stok < 0:
                print("Error: Harga dan stok harus positif.")
                return

            # Membuat kode barang unik berdasarkan timestamp
            kode_barang = "BRG" + str(int(time.time()))

            self.inventori[kode_barang] = {
                'nama': nama,
                'harga': harga,
                'stok': stok
            }
            self.simpan_data()
            print(f"\n[Sukses] Barang '{nama}' dengan kode '{kode_barang}' berhasil ditambahkan.")
        except ValueError: # Exception Handling untuk input non-angka [cite: 32, 44]
            print("\n[Error] Harga dan stok harus berupa angka.")

    def lihat_semua_barang(self, urutkan=False):
        """
        Menampilkan semua barang di inventori.
        Menggunakan Bubble Sort untuk mengurutkan jika diminta.
        """
        if not self.inventori:
            print("Inventori kosong.")
            return

        # Mengubah dictionary menjadi list of tuples untuk diurutkan
        daftar_barang = list(self.inventori.items())

        if urutkan:
            # Implementasi Bubble Sort
            n = len(daftar_barang)
            for i in range(n):
                for j in range(0, n - i - 1):
                    # Urutkan berdasarkan nama barang (A-Z)
                    if daftar_barang[j][1]['nama'] > daftar_barang[j+1][1]['nama']:
                        daftar_barang[j], daftar_barang[j+1] = daftar_barang[j+1], daftar_barang[j]
            print("--- Daftar Barang (Diurutkan Berdasarkan Nama) ---")
        else:
            print("--- Daftar Barang ---")

        for kode, detail in daftar_barang:
            print(f"Kode: {kode} | Nama: {detail['nama']} | Harga: Rp{detail['harga']:,} | Stok: {detail['stok']}")
        print("-" * 20)

    def cari_barang(self):
        """
        Mencari barang berdasarkan nama (pencarian linear). [cite: 25, 38]
        """
        if not self.inventori:
            print("Inventori kosong.")
            return

        keyword = input("Masukkan nama barang yang dicari: ").lower()
        ditemukan = False
        print("\n--- Hasil Pencarian ---")
        # Implementasi Linear Search
        for kode, detail in self.inventori.items():
            if keyword in detail['nama'].lower():
                print(f"Kode: {kode} | Nama: {detail['nama']} | Harga: Rp{detail['harga']:,} | Stok: {detail['stok']}")
                ditemukan = True

        if not ditemukan:
            print("Barang tidak ditemukan.")
        print("-" * 20)


    def update_stok(self):
        """
        Mengupdate stok barang berdasarkan kode barang. [cite: 39]
        """
        kode_barang = input("Masukkan kode barang yang akan diupdate: ").upper()
        if kode_barang in self.inventori:
            try:
                stok_baru = int(input(f"Masukkan jumlah stok baru untuk '{self.inventori[kode_barang]['nama']}': "))
                if stok_baru < 0:
                    print("Error: Stok tidak boleh negatif.")
                    return
                self.inventori[kode_barang]['stok'] = stok_baru
                self.simpan_data()
                print("[Sukses] Stok berhasil diupdate.")
            except ValueError:
                print("\n[Error] Stok harus berupa angka.")
        else:
            print("[Error] Kode barang tidak ditemukan.")

    def hapus_barang(self):
        """
        Menghapus barang dari inventori. [cite: 40]
        """
        kode_barang = input("Masukkan kode barang yang akan dihapus: ").upper()
        if kode_barang in self.inventori:
            nama_barang = self.inventori[kode_barang]['nama']
            konfirmasi = input(f"Apakah Anda yakin ingin menghapus '{nama_barang}' (y/n)? ").lower()
            if konfirmasi == 'y':
                del self.inventori[kode_barang]
                self.simpan_data()
                print(f"[Sukses] Barang '{nama_barang}' berhasil dihapus.")
            else:
                print("Penghapusan dibatalkan.")
        else:
            print("[Error] Kode barang tidak ditemukan.")

    # ===============================================
    # FUNGSI-FUNGSI PELANGGAN
    # ===============================================
    def belanja(self, keranjang):
        """
        Proses pelanggan memilih barang untuk dimasukkan ke keranjang.
        """
        self.lihat_semua_barang(urutkan=True)
        if not self.inventori:
            return keranjang

        while True:
            kode_barang = input("Masukkan kode barang yang ingin dibeli (atau 'selesai' untuk ke kasir): ").upper()
            if kode_barang.lower() == 'selesai':
                break

            if kode_barang in self.inventori:
                try:
                    jumlah = int(input(f"Berapa banyak '{self.inventori[kode_barang]['nama']}' yang ingin dibeli? "))
                    if jumlah <= 0:
                        print("Jumlah harus lebih dari 0.")
                    elif jumlah > self.inventori[kode_barang]['stok']:
                        print(f"Maaf, stok tidak mencukupi. Stok tersisa: {self.inventori[kode_barang]['stok']}")
                    else:
                        # Tambahkan ke keranjang
                        if kode_barang in keranjang:
                            keranjang[kode_barang]['jumlah'] += jumlah
                        else:
                            keranjang[kode_barang] = {
                                'nama': self.inventori[kode_barang]['nama'],
                                'harga': self.inventori[kode_barang]['harga'],
                                'jumlah': jumlah
                            }
                        print(f"[+] {jumlah} x '{self.inventori[kode_barang]['nama']}' ditambahkan ke keranjang.")
                except ValueError:
                    print("[Error] Jumlah harus berupa angka.")
            else:
                print("[Error] Kode barang tidak valid.")
        return keranjang

    def checkout(self, keranjang):
        """
        Proses pembayaran dan pembaruan stok inventori.
        """
        if not keranjang:
            print("Keranjang Anda kosong.")
            return

        print("\n--- Keranjang Belanja Anda ---")
        total_harga = 0
        for kode, item in keranjang.items():
            subtotal = item['harga'] * item['jumlah']
            total_harga += subtotal
            print(f"{item['nama']} ({item['jumlah']} x Rp{item['harga']:,}) = Rp{subtotal:,}")
        print("-" * 30)
        print(f"Total Belanja: Rp{total_harga:,}")

        konfirmasi = input("Lanjutkan ke pembayaran (y/n)? ").lower()
        if konfirmasi == 'y':
            # Update stok di inventori
            for kode, item in keranjang.items():
                self.inventori[kode]['stok'] -= item['jumlah']
            self.simpan_data()
            print("\nTerima kasih telah berbelanja! Stok inventori telah diperbarui.")
            keranjang.clear()
        else:
            print("Checkout dibatalkan.")

# ===============================================
# FUNGSI-FUNGSI TAMPILAN MENU (INTERFACE) [cite: 43]
# ===============================================

def menu_admin(toko):
    """Menampilkan menu untuk admin."""
    while True:
        print("\n--- Menu Admin ---")
        print("1. Tambah Barang Baru")
        print("2. Lihat Semua Barang (Default)")
        print("3. Lihat Semua Barang (Urutkan A-Z)")
        print("4. Cari Barang")
        print("5. Update Stok Barang")
        print("6. Hapus Barang")
        print("7. Kembali ke Menu Utama")
        pilihan = input("Pilih opsi: ")

        if pilihan == '1':
            toko.tambah_barang()
        elif pilihan == '2':
            toko.lihat_semua_barang()
        elif pilihan == '3':
            toko.lihat_semua_barang(urutkan=True)
        elif pilihan == '4':
            toko.cari_barang()
        elif pilihan == '5':
            toko.update_stok()
        elif pilihan == '6':
            toko.hapus_barang()
        elif pilihan == '7':
            break
        else:
            print("Pilihan tidak valid.")
        input("\nTekan Enter untuk melanjutkan...")
        os.system('cls' if os.name == 'nt' else 'clear')


def menu_pelanggan(toko):
    """Menampilkan menu untuk pelanggan."""
    keranjang_belanja = {} # Dictionary untuk menyimpan keranjang
    while True:
        print("\n--- Menu Pelanggan ---")
        print("1. Lihat & Beli Barang")
        print("2. Lihat Keranjang & Checkout")
        print("3. Kembali ke Menu Utama")
        pilihan = input("Pilih opsi: ")

        if pilihan == '1':
            keranjang_belanja = toko.belanja(keranjang_belanja)
        elif pilihan == '2':
            toko.checkout(keranjang_belanja)
            # Jika checkout berhasil, keranjang akan kosong dan loop akan lanjut
            if not keranjang_belanja: # kembali ke menu jika checkout sukses
                 break
        elif pilihan == '3':
            break
        else:
            print("Pilihan tidak valid.")
        input("\nTekan Enter untuk melanjutkan...")
        os.system('cls' if os.name == 'nt' else 'clear')


def main():
    """Fungsi utama untuk menjalankan aplikasi."""
    toko = TokoOnline(NAMA_FILE)
    while True:
        os.system('cls' if os.name == 'nt' else 'clear')
        print("========================================")
        print(" Selamat Datang di Aplikasi Toko Online ")
        print("========================================")
        print("Pilih peran Anda:")
        print("1. Admin")
        print("2. Pelanggan")
        print("3. Keluar")
        peran = input("Pilih opsi: ")

        if peran == '1':
            menu_admin(toko)
        elif peran == '2':
            menu_pelanggan(toko)
        elif peran == '3':
            print("Terima kasih telah menggunakan aplikasi ini.")
            break
        else:
            print("Pilihan tidak valid. Coba lagi.")
            time.sleep(1)


if __name__ == "__main__":
    main()

 Selamat Datang di Aplikasi Toko Online 
Pilih peran Anda:
1. Admin
2. Pelanggan
3. Keluar
Pilih opsi: 1

--- Menu Admin ---
1. Tambah Barang Baru
2. Lihat Semua Barang (Default)
3. Lihat Semua Barang (Urutkan A-Z)
4. Cari Barang
5. Update Stok Barang
6. Hapus Barang
7. Kembali ke Menu Utama
Pilih opsi: 1
Masukkan nama barang: buku
Masukkan harga barang: 3000
Masukkan jumlah stok: 20

[Sukses] Barang 'buku' dengan kode 'BRG1752482387' berhasil ditambahkan.

Tekan Enter untuk melanjutkan...1

--- Menu Admin ---
1. Tambah Barang Baru
2. Lihat Semua Barang (Default)
3. Lihat Semua Barang (Urutkan A-Z)
4. Cari Barang
5. Update Stok Barang
6. Hapus Barang
7. Kembali ke Menu Utama
Pilih opsi: 1
Masukkan nama barang: pensil 2b
Masukkan harga barang: 2000
Masukkan jumlah stok: 32

[Sukses] Barang 'pensil 2b' dengan kode 'BRG1752482420' berhasil ditambahkan.

Tekan Enter untuk melanjutkan...1

--- Menu Admin ---
1. Tambah Barang Baru
2. Lihat Semua Barang (Default)
3. Lihat Semua Barang (Urutka